In [1]:
import pandas as pd
import numpy as np
import nbimporter
from datetime import datetime, timedelta
import glob
import os
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:

def correlation_matrix(endresult_df, categorical_columns, cut_off_limit):
    columns_to_keep = [col for col in endresult_df.columns if not any(excl in col for excl in categorical_columns)]

    df_relevant = endresult_df[columns_to_keep]  
    correlation_matrix = df_relevant.corr()
    
    # Set the size of the figure
    plt.figure(figsize=(10, 8))
    
    # Create the heat map
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
    
    # Add a title
    plt.title('Correlation Matrix Heatmap')
    
    # Show the plot
    plt.show()
    
    target_column = "Gesamt (Netzlast) [MWh] Originalauflösungen"
    
    columns_to_drop = correlation_matrix.index[abs(correlation_matrix[target_column]) < cut_off_limit].tolist()
    
    # Drop those columns from df_relevant
    df_filtered = df_relevant.drop(columns=columns_to_drop)
    #print(df_filtered)
    # If you want to replot the correlation matrix for the filtered dataframe
    filtered_correlation_matrix = df_filtered.corr()
    
    plt.figure(figsize=(10, 8))
    
    # Create the heatmap
    sns.heatmap(filtered_correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
    
    # Add a title
    plt.title('Filtered Correlation Matrix Heatmap')
    
    # Show the plot
    plt.show()

    df_end = endresult_df.drop(columns=columns_to_drop)
    return df_end